In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
url = 'https://raw.githubusercontent.com/dandia14/project-capstone-team-csd-134/ml/dataset/diabetes.csv'
diabetes = pd.read_csv(url)
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
cols = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
for col in cols:
    diabetes[col].replace(0,np.NaN,inplace=True)

In [4]:
# Mengganti nilai NaN dengan nilai median sesuai target Outcome
for col in diabetes.columns:
    diabetes.loc[(diabetes["Outcome"]==0) & (diabetes[col].isnull()),col] = diabetes[diabetes["Outcome"]==0][col].median()
    diabetes.loc[(diabetes["Outcome"]==1) & (diabetes[col].isnull()),col] = diabetes[diabetes["Outcome"]==1][col].median()

In [5]:
Q1 = diabetes.quantile(0.25)
Q3 = diabetes.quantile(0.75)
IQR=Q3-Q1
diabetes=diabetes[~((diabetes<(Q1-1.5*IQR))|(diabetes>(Q3+1.5*IQR))).any(axis=1)]
 
# Cek ukuran dataset setelah kita drop outliers
diabetes.shape

(590, 9)

In [6]:
X = diabetes.values
X = np.delete(X, 8, axis=1)
y = diabetes['Outcome']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
rf_Model = RandomForestClassifier(n_estimators= 50, max_depth= 4,
                                  max_features= 'log2', min_samples_leaf= 1,
                                  bootstrap= True, random_state=0)

In [10]:
rf_Model.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, max_features='log2', n_estimators=50,
                       random_state=0)

In [11]:
print(f'Train Accuracy - : {rf_Model.score(X_train, y_train):.3f}')
print(f'Test Accuracy - : {rf_Model.score(X_test, y_test):.3f}')

Train Accuracy - : 0.943
Test Accuracy - : 0.898


In [12]:
pickle.dump(rf_Model, open('model_rfc.pkl', 'wb'))

In [13]:
location = './model_rfc.pkl'
file = open(location, "rb")
model = pickle.load(file)

In [14]:
data1 = [10,	168,	74,	32,	169.5,	38,	0.537,	34]
data2 = [4,	110,	92,	27,	102.5,	37.6,	0.191,	30]
data1 = np.array(data1)
data2 = np.array(data2)

data1 = data1.reshape(1,-1)
data2 = data2.reshape(1,-1)
print(data1)
print(data2)

[[ 10.    168.     74.     32.    169.5    38.      0.537  34.   ]]
[[  4.    110.     92.     27.    102.5    37.6     0.191  30.   ]]


In [15]:
prediction1 = model.predict(data1)
prediction2 = model.predict(data2)

print(prediction1)
print(prediction2)

[1]
[0]
